In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px

import glob

from locations import *
from parsing import *

In [3]:
df = pd.read_csv('madrid_parsed.csv')
df

,price,title,location,lat,lng,Referencia,Clasificación,Baños,Conservación,Superficie construida,Habitaciones
0,176000.0,Apartamento en venta en Abantos-Monte Carmelo-...,Abantos-Monte Carmelo-El Rosario (San Lorenzo ...,40.593445,-4.145386,4994318-VF-812,En trámite,1.0,A estrenar,63 m²,1.0
1,116500.0,"Apartamento en venta en Calle de Ercilla, 20, ...","Calle de Ercilla, 20, cerca de Calle del Labra...",40.402079,-3.702151,EP805-1309,En trámite,1.0,NaN,25 m²,NaN
2,169000.0,Apartamento en venta en Ajalvir,Ajalvir,40.534457,-3.479415,IFC76153-1690TN,En trámite,1.0,En buen estado,70 m²,1.0
3,174000.0,Apartamento en venta en Alcorcón,Casco Antiguo (Alcorcón),40.344358,-3.825283,SA1398-4340/6592,No indicado,1.0,NaN,60 m²,1.0
4,130000.0,Apartamento en venta en Alcorcón,Casco Antiguo (Alcorcón),40.347096,-3.827826,SA2135-04930/7619,No indicado,1.0,NaN,60 m²,2.0
...,...,...,...,...,...,...,...,...,...,...,...
15832,135000.0,Piso en venta en Calle de la Circunvalación,Calle de la Circunvalación. Zona Suroeste (Tor...,40.454606,-3.455234,SA4463-BANCO-0001,No indicado,1.0,A reformar,67 m²,3.0
15833,210000.0,Piso en venta en Calle de Urano,Calle de Urano. Zona Suroeste (Torrejón de Ardoz),40.441928,-3.473036,SA3697-IL-04891,En trámite,2.0,En buen estado,79 m²,1.0
15834,324600.0,Piso en venta en Fresnos,Zona Suroeste (Torrejón de Ardoz),40.448403,-3.470899,SA555-39508999/1670,No indicado,3.0,NaN,140 m²,4.0
15835,249900.0,Piso en venta en Calle de Cibeles,Calle de Cibeles. Zona Suroeste (Torrejón de A...,40.444707,-3.473520,4525678-000021,En trámite,2.0,En buen estado,106 m²,3.0
